# 📝 Lezione 28 — Progetto End-to-End: Unsupervised Learning

---

## 🎯 Obiettivo del Progetto

> **CASO STUDIO**: Segmentazione clienti per un e-commerce

Applicheremo TUTTE le tecniche apprese nel blocco Unsupervised Learning (Lezioni 19-27) in un progetto completo.

---

## 📚 Tecniche Utilizzate

| Lezione | Tecnica | Uso nel Progetto |
|---------|---------|------------------|
| 19 | Unsupervised Intro | Comprensione problema |
| 20 | K-Means | Segmentazione primaria |
| 21 | Scelta K | Numero segmenti ottimale |
| 22 | Clustering Gerarchico | Validazione alternativa |
| 23 | DBSCAN | Identificazione outlier naturali |
| 24 | PCA | Riduzione dimensionalità |
| 25 | PCA + Clustering | Visualizzazione segmenti |
| 26 | Anomaly Detection | Clienti anomali |
| 27 | Feature Engineering | Arricchimento dati |

---

## 🏗️ Struttura del Progetto

1. **Business Understanding** — Definizione del problema
2. **Data Loading & Exploration** — Caricamento e EDA
3. **Data Preprocessing** — Pulizia e preparazione
4. **Feature Engineering** — Creazione nuove features
5. **Dimensionality Reduction** — PCA
6. **Clustering** — Segmentazione
7. **Anomaly Detection** — Identificazione outlier
8. **Interpretation** — Interpretazione business
9. **Deliverables** — Output finali

---

# 📖 FASE 1: Business Understanding

## 🎯 Il Problema

Un e-commerce vuole **segmentare i propri clienti** per:
- 📧 Marketing personalizzato
- 💰 Ottimizzazione budget advertising
- 🎁 Offerte mirate per segmento
- ⚠️ Identificazione clienti a rischio churn

## 📋 Domande Business

1. **Quanti segmenti** di clienti abbiamo?
2. **Come si caratterizzano** i segmenti?
3. **Chi sono i clienti anomali**?
4. **Quale strategia** per ogni segmento?

## 📊 Dati Disponibili

Simuleremo un dataset RFM (Recency, Frequency, Monetary) con metriche comportamentali:

| Feature | Descrizione |
|---------|-------------|
| `recency` | Giorni dall'ultimo acquisto |
| `frequency` | Numero di acquisti negli ultimi 12 mesi |
| `monetary` | Spesa totale negli ultimi 12 mesi |
| `avg_basket` | Valore medio del carrello |
| `tenure` | Mesi da cliente |
| `online_ratio` | % acquisti online vs negozio |

In [ ]:
# =============================================================================
# 📊 FASE 2: Data Loading & Exploration
# =============================================================================
"""
OBIETTIVO: Caricare i dati e fare una prima esplorazione
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configurazione
np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

print("=" * 70)
print("📊 FASE 2: DATA LOADING & EXPLORATION")
print("=" * 70)

# ═══════════════════════════════════════════════════════════════════════════
# 2.1 CREAZIONE DATASET SIMULATO
# ═══════════════════════════════════════════════════════════════════════════
print("\n📦 Creazione dataset clienti e-commerce simulato...")

n_customers = 1000

# Simulo 4 segmenti naturali di clienti
segments = {
    'VIP': {'n': 100, 'recency': (5, 10), 'frequency': (20, 30), 'monetary': (5000, 10000)},
    'Regular': {'n': 350, 'recency': (15, 30), 'frequency': (8, 15), 'monetary': (1000, 3000)},
    'Occasional': {'n': 350, 'recency': (45, 90), 'frequency': (2, 5), 'monetary': (200, 800)},
    'Churned': {'n': 200, 'recency': (180, 365), 'frequency': (1, 3), 'monetary': (50, 300)}
}

data_list = []
for seg_name, params in segments.items():
    n = params['n']
    data_list.append(pd.DataFrame({
        'customer_id': range(len(data_list) * 1000, len(data_list) * 1000 + n),
        'recency': np.random.uniform(*params['recency'], n),
        'frequency': np.random.uniform(*params['frequency'], n),
        'monetary': np.random.uniform(*params['monetary'], n),
        'avg_basket': np.random.uniform(30, 150, n),
        'tenure': np.random.uniform(6, 60, n),
        'online_ratio': np.random.uniform(0.2, 0.9, n),
        '_true_segment': seg_name  # Solo per validazione, non lo useremo
    }))

df = pd.concat(data_list, ignore_index=True)

# Aggiungo un po' di rumore e outlier
noise_idx = np.random.choice(len(df), size=50, replace=False)
df.loc[noise_idx[:20], 'monetary'] *= 3  # Super spenditori
df.loc[noise_idx[20:35], 'recency'] = np.random.uniform(400, 500, 15)  # Super inattivi
df.loc[noise_idx[35:], 'frequency'] *= 0.1  # Frequenza anomala

# Shuffle
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"✅ Dataset creato: {df.shape[0]} clienti, {df.shape[1]} features")

# ═══════════════════════════════════════════════════════════════════════════
# 2.2 PRIMA ESPLORAZIONE
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("📋 STRUTTURA DATASET")
print("-" * 70)

print(f"\n📐 Shape: {df.shape}")
print(f"\n📊 Tipi dati:")
print(df.dtypes)

print(f"\n📈 Statistiche descrittive:")
print(df.describe().round(2))

# Features da usare per clustering (escludo customer_id e _true_segment)
features_to_use = ['recency', 'frequency', 'monetary', 'avg_basket', 'tenure', 'online_ratio']
df_features = df[features_to_use].copy()

# ═══════════════════════════════════════════════════════════════════════════
# 2.3 VISUALIZZAZIONE EDA
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

for idx, col in enumerate(features_to_use):
    ax = axes[idx // 3, idx % 3]
    ax.hist(df[col], bins=30, color='steelblue', alpha=0.7, edgecolor='black')
    ax.axvline(df[col].mean(), color='red', linestyle='--', label=f'Mean: {df[col].mean():.1f}')
    ax.axvline(df[col].median(), color='green', linestyle='--', label=f'Median: {df[col].median():.1f}')
    ax.set_xlabel(col)
    ax.set_ylabel('Frequenza')
    ax.set_title(f'📊 Distribuzione {col}')
    ax.legend(fontsize=8)

plt.tight_layout()
plt.suptitle('📊 EDA: Distribuzione Features', y=1.02, fontsize=14)
plt.show()

# Correlation matrix
fig, ax = plt.subplots(figsize=(8, 6))
corr_matrix = df_features.corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, ax=ax, square=True)
ax.set_title('🔗 Matrice di Correlazione')
plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("✅ FASE 2 COMPLETATA!")
print("=" * 70)

In [ ]:
# =============================================================================
# 🔧 FASE 3: Data Preprocessing
# =============================================================================
"""
OBIETTIVO: Pulire e preparare i dati per il clustering
"""

from sklearn.preprocessing import StandardScaler, RobustScaler

print("=" * 70)
print("🔧 FASE 3: DATA PREPROCESSING")
print("=" * 70)

# ═══════════════════════════════════════════════════════════════════════════
# 3.1 CHECK VALORI MANCANTI
# ═══════════════════════════════════════════════════════════════════════════
print("\n📋 Check valori mancanti:")
missing = df_features.isnull().sum()
print(missing)
print(f"\n✅ Nessun valore mancante!" if missing.sum() == 0 else "⚠️ Valori mancanti presenti!")

# ═══════════════════════════════════════════════════════════════════════════
# 3.2 CHECK OUTLIER CON IQR
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("📊 CHECK OUTLIER (metodo IQR)")
print("-" * 70)

def count_outliers_iqr(series, k=1.5):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - k * IQR
    upper = Q3 + k * IQR
    return ((series < lower) | (series > upper)).sum()

print("\n📈 Outlier per feature:")
for col in features_to_use:
    n_outliers = count_outliers_iqr(df_features[col])
    pct = n_outliers / len(df_features) * 100
    print(f"  {col}: {n_outliers} ({pct:.1f}%)")

# ═══════════════════════════════════════════════════════════════════════════
# 3.3 SCALING
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("📏 SCALING")
print("-" * 70)

# Uso RobustScaler per robustezza agli outlier
scaler = RobustScaler()
X_scaled = scaler.fit_transform(df_features)
df_scaled = pd.DataFrame(X_scaled, columns=features_to_use)

print("\n📊 Statistiche dopo RobustScaler:")
print(df_scaled.describe().round(3))

# ═══════════════════════════════════════════════════════════════════════════
# 3.4 VISUALIZZAZIONE PRIMA/DOPO SCALING
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Prima
ax1 = axes[0]
df_features.boxplot(ax=ax1)
ax1.set_title('📊 Distribuzione PRIMA dello Scaling')
ax1.set_xticklabels(features_to_use, rotation=45, ha='right')

# Dopo
ax2 = axes[1]
df_scaled.boxplot(ax=ax2)
ax2.set_title('📊 Distribuzione DOPO RobustScaler')
ax2.set_xticklabels(features_to_use, rotation=45, ha='right')

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("✅ FASE 3 COMPLETATA!")
print("=" * 70)

In [ ]:
# =============================================================================
# 🔍 FASE 4: Dimensionality Reduction (PCA)
# =============================================================================
"""
OBIETTIVO: Ridurre dimensionalità e visualizzare i dati
"""

from sklearn.decomposition import PCA

print("=" * 70)
print("🔍 FASE 4: DIMENSIONALITY REDUCTION (PCA)")
print("=" * 70)

# ═══════════════════════════════════════════════════════════════════════════
# 4.1 PCA COMPLETA PER ANALISI
# ═══════════════════════════════════════════════════════════════════════════
pca_full = PCA()
pca_full.fit(X_scaled)

# Varianza spiegata
print("\n📈 Varianza spiegata per componente:")
cumsum = np.cumsum(pca_full.explained_variance_ratio_)
for i, (var, cum) in enumerate(zip(pca_full.explained_variance_ratio_, cumsum)):
    print(f"  PC{i+1}: {var:.1%} (cumulata: {cum:.1%})")

# ═══════════════════════════════════════════════════════════════════════════
# 4.2 SCELTA NUMERO COMPONENTI
# ═══════════════════════════════════════════════════════════════════════════
# Scelgo componenti che spiegano almeno 80% varianza
n_components_80 = np.argmax(cumsum >= 0.80) + 1
n_components_90 = np.argmax(cumsum >= 0.90) + 1

print(f"\n📊 Componenti necessarie:")
print(f"  Per 80% varianza: {n_components_80}")
print(f"  Per 90% varianza: {n_components_90}")

# Per visualizzazione uso 2, per clustering uso 3
pca_2d = PCA(n_components=2)
pca_3d = PCA(n_components=3)

X_pca_2d = pca_2d.fit_transform(X_scaled)
X_pca_3d = pca_3d.fit_transform(X_scaled)

print(f"\n✅ PCA 2D: {pca_2d.explained_variance_ratio_.sum():.1%} varianza spiegata")
print(f"✅ PCA 3D: {pca_3d.explained_variance_ratio_.sum():.1%} varianza spiegata")

# ═══════════════════════════════════════════════════════════════════════════
# 4.3 ANALISI LOADINGS
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("📊 LOADINGS (importanza features per componente)")
print("-" * 70)

loadings = pd.DataFrame(
    pca_2d.components_.T,
    columns=['PC1', 'PC2'],
    index=features_to_use
)
print(loadings.round(3))

print("\n📝 Interpretazione:")
pc1_top = loadings['PC1'].abs().idxmax()
pc2_top = loadings['PC2'].abs().idxmax()
print(f"  PC1 dominato da: {pc1_top}")
print(f"  PC2 dominato da: {pc2_top}")

# ═══════════════════════════════════════════════════════════════════════════
# 4.4 VISUALIZZAZIONE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot 1: Scree plot
ax1 = axes[0]
ax1.bar(range(1, len(pca_full.explained_variance_ratio_)+1), 
        pca_full.explained_variance_ratio_, alpha=0.7, label='Individuale')
ax1.plot(range(1, len(cumsum)+1), cumsum, 'ro-', label='Cumulata')
ax1.axhline(y=0.8, color='g', linestyle='--', label='80%')
ax1.axhline(y=0.9, color='orange', linestyle='--', label='90%')
ax1.set_xlabel('Componente')
ax1.set_ylabel('Varianza Spiegata')
ax1.set_title('📊 Scree Plot')
ax1.legend()

# Plot 2: Dati in spazio PCA 2D
ax2 = axes[1]
ax2.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], alpha=0.5, s=20, c='steelblue')
ax2.set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.1%})')
ax2.set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.1%})')
ax2.set_title('📍 Clienti in Spazio PCA 2D')

# Plot 3: Loadings biplot
ax3 = axes[2]
ax3.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], alpha=0.2, s=10, c='gray')
# Aggiungo vettori loadings
scale = 3  # Fattore di scala per visualizzazione
for i, feature in enumerate(features_to_use):
    ax3.arrow(0, 0, loadings.loc[feature, 'PC1']*scale, loadings.loc[feature, 'PC2']*scale,
              head_width=0.1, head_length=0.05, fc='red', ec='red')
    ax3.text(loadings.loc[feature, 'PC1']*scale*1.1, loadings.loc[feature, 'PC2']*scale*1.1,
             feature, fontsize=9, ha='center')
ax3.set_xlabel('PC1')
ax3.set_ylabel('PC2')
ax3.set_title('📊 Biplot: Loadings + Dati')
ax3.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax3.axvline(x=0, color='k', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("✅ FASE 4 COMPLETATA!")
print("=" * 70)

In [ ]:
# =============================================================================
# 🎯 FASE 5: Scelta del Numero di Cluster
# =============================================================================
"""
OBIETTIVO: Determinare il numero ottimale di segmenti
"""

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

print("=" * 70)
print("🎯 FASE 5: SCELTA DEL NUMERO DI CLUSTER")
print("=" * 70)

# ═══════════════════════════════════════════════════════════════════════════
# 5.1 METODO DEL GOMITO (ELBOW)
# ═══════════════════════════════════════════════════════════════════════════
print("\n📊 Calcolo metriche per K = 2 a 10...")

k_range = range(2, 11)
inertias = []
silhouettes = []
calinski = []
davies = []

for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_scaled)
    
    inertias.append(km.inertia_)
    silhouettes.append(silhouette_score(X_scaled, labels))
    calinski.append(calinski_harabasz_score(X_scaled, labels))
    davies.append(davies_bouldin_score(X_scaled, labels))
    
    print(f"  K={k}: Silhouette={silhouettes[-1]:.3f}, Inertia={inertias[-1]:.0f}")

# ═══════════════════════════════════════════════════════════════════════════
# 5.2 IDENTIFICAZIONE K OTTIMALE
# ═══════════════════════════════════════════════════════════════════════════
best_k_silhouette = k_range[np.argmax(silhouettes)]
best_k_calinski = k_range[np.argmax(calinski)]
best_k_davies = k_range[np.argmin(davies)]

print(f"\n📊 K ottimale secondo diverse metriche:")
print(f"  Silhouette (max): K = {best_k_silhouette}")
print(f"  Calinski-Harabasz (max): K = {best_k_calinski}")
print(f"  Davies-Bouldin (min): K = {best_k_davies}")

# Decisione finale (considerando anche il business context: 4 segmenti ha senso)
k_optimal = 4
print(f"\n✅ Scelta finale: K = {k_optimal} (bilanciando metriche e interpretabilità)")

# ═══════════════════════════════════════════════════════════════════════════
# 5.3 VISUALIZZAZIONE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Elbow
ax1 = axes[0, 0]
ax1.plot(list(k_range), inertias, 'bo-', linewidth=2)
ax1.axvline(x=k_optimal, color='red', linestyle='--', label=f'K scelto = {k_optimal}')
ax1.set_xlabel('Numero di Cluster (K)')
ax1.set_ylabel('Inertia')
ax1.set_title('📊 Metodo del Gomito (Elbow)')
ax1.legend()

# Silhouette
ax2 = axes[0, 1]
ax2.plot(list(k_range), silhouettes, 'go-', linewidth=2)
ax2.axvline(x=k_optimal, color='red', linestyle='--', label=f'K scelto = {k_optimal}')
ax2.set_xlabel('Numero di Cluster (K)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('📊 Silhouette Score (più alto = meglio)')
ax2.legend()

# Calinski-Harabasz
ax3 = axes[1, 0]
ax3.plot(list(k_range), calinski, 'mo-', linewidth=2)
ax3.axvline(x=k_optimal, color='red', linestyle='--', label=f'K scelto = {k_optimal}')
ax3.set_xlabel('Numero di Cluster (K)')
ax3.set_ylabel('Calinski-Harabasz Score')
ax3.set_title('📊 Calinski-Harabasz (più alto = meglio)')
ax3.legend()

# Davies-Bouldin
ax4 = axes[1, 1]
ax4.plot(list(k_range), davies, 'co-', linewidth=2)
ax4.axvline(x=k_optimal, color='red', linestyle='--', label=f'K scelto = {k_optimal}')
ax4.set_xlabel('Numero di Cluster (K)')
ax4.set_ylabel('Davies-Bouldin Score')
ax4.set_title('📊 Davies-Bouldin (più basso = meglio)')
ax4.legend()

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("✅ FASE 5 COMPLETATA!")
print("=" * 70)

In [ ]:
# =============================================================================
# 🎯 FASE 6: Clustering (K-Means)
# =============================================================================
"""
OBIETTIVO: Segmentare i clienti con K-Means
"""

from scipy.spatial.distance import cdist

print("=" * 70)
print("🎯 FASE 6: CLUSTERING (K-MEANS)")
print("=" * 70)

# ═══════════════════════════════════════════════════════════════════════════
# 6.1 K-MEANS FINALE
# ═══════════════════════════════════════════════════════════════════════════
kmeans = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)

# Aggiungo al dataframe
df['cluster'] = cluster_labels
df_features['cluster'] = cluster_labels
df_scaled['cluster'] = cluster_labels

print(f"\n📊 Distribuzione cluster:")
cluster_counts = df['cluster'].value_counts().sort_index()
for c, count in cluster_counts.items():
    print(f"  Cluster {c}: {count} clienti ({count/len(df)*100:.1f}%)")

# ═══════════════════════════════════════════════════════════════════════════
# 6.2 PROFILING DEI CLUSTER
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("📊 PROFILING DEI CLUSTER")
print("-" * 70)

# Medie per cluster
cluster_profiles = df_features.groupby('cluster')[features_to_use].mean()
print("\n📈 Valori medi per cluster:")
print(cluster_profiles.round(2))

# Normalizzato rispetto alla media globale
global_means = df_features[features_to_use].mean()
cluster_profiles_norm = (cluster_profiles - global_means) / global_means * 100
print("\n📈 Differenza % dalla media globale:")
print(cluster_profiles_norm.round(1))

# ═══════════════════════════════════════════════════════════════════════════
# 6.3 NAMING DEI CLUSTER (INTERPRETAZIONE BUSINESS)
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("🏷️ NAMING DEI CLUSTER")
print("-" * 70)

# Ordino cluster per monetary (valore cliente)
cluster_order = cluster_profiles['monetary'].sort_values(ascending=False).index.tolist()

cluster_names = {}
name_mapping = ['🌟 VIP', '💼 Regular', '🎯 Occasional', '⚠️ At Risk']

for i, cluster_id in enumerate(cluster_order):
    cluster_names[cluster_id] = name_mapping[i]
    profile = cluster_profiles.loc[cluster_id]
    print(f"\n{name_mapping[i]} (Cluster {cluster_id}):")
    print(f"  Recency: {profile['recency']:.1f} giorni")
    print(f"  Frequency: {profile['frequency']:.1f} acquisti")
    print(f"  Monetary: €{profile['monetary']:.0f}")
    print(f"  Clienti: {cluster_counts[cluster_id]}")

# Aggiungo nomi al dataframe
df['segment_name'] = df['cluster'].map(cluster_names)

# ═══════════════════════════════════════════════════════════════════════════
# 6.4 VISUALIZZAZIONE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Plot 1: Cluster in spazio PCA
ax1 = axes[0, 0]
colors = ['gold', 'steelblue', 'green', 'red']
for c in range(k_optimal):
    mask = cluster_labels == c
    ax1.scatter(X_pca_2d[mask, 0], X_pca_2d[mask, 1], 
                c=colors[c], label=cluster_names[c], alpha=0.6, s=30)
# Centroidi
centroids_pca = pca_2d.transform(kmeans.cluster_centers_)
ax1.scatter(centroids_pca[:, 0], centroids_pca[:, 1], 
            c='black', marker='X', s=200, edgecolors='white', linewidths=2)
ax1.set_xlabel('PC1')
ax1.set_ylabel('PC2')
ax1.set_title('🎯 Segmenti Clienti in Spazio PCA')
ax1.legend()

# Plot 2: Radar chart profili
ax2 = axes[0, 1]
categories = features_to_use
N = len(categories)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Normalizzo per radar (0-1)
cluster_profiles_radar = (cluster_profiles - cluster_profiles.min()) / (cluster_profiles.max() - cluster_profiles.min())

for c in range(k_optimal):
    values = cluster_profiles_radar.loc[c].values.tolist()
    values += values[:1]
    ax2.plot(angles, values, 'o-', linewidth=2, label=cluster_names[c], color=colors[c])
    ax2.fill(angles, values, alpha=0.1, color=colors[c])

ax2.set_xticks(angles[:-1])
ax2.set_xticklabels(categories, fontsize=9)
ax2.set_title('📊 Profili Cluster (Radar Chart)')
ax2.legend(loc='upper right', bbox_to_anchor=(1.3, 1))

# Plot 3: Box plot monetary per cluster
ax3 = axes[1, 0]
df.boxplot(column='monetary', by='segment_name', ax=ax3)
ax3.set_xlabel('Segmento')
ax3.set_ylabel('Monetary (€)')
ax3.set_title('💰 Distribuzione Monetary per Segmento')
plt.suptitle('')

# Plot 4: Scatter frequency vs monetary
ax4 = axes[1, 1]
for c in range(k_optimal):
    mask = df['cluster'] == c
    ax4.scatter(df.loc[mask, 'frequency'], df.loc[mask, 'monetary'],
                c=colors[c], label=cluster_names[c], alpha=0.5, s=30)
ax4.set_xlabel('Frequency (acquisti)')
ax4.set_ylabel('Monetary (€)')
ax4.set_title('📊 Frequency vs Monetary per Segmento')
ax4.legend()

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("✅ FASE 6 COMPLETATA!")
print("=" * 70)

In [ ]:
# =============================================================================
# 🔍 FASE 7: Anomaly Detection
# =============================================================================
"""
OBIETTIVO: Identificare clienti anomali all'interno di ogni segmento
"""

from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

print("=" * 70)
print("🔍 FASE 7: ANOMALY DETECTION")
print("=" * 70)

# ═══════════════════════════════════════════════════════════════════════════
# 7.1 ISOLATION FOREST GLOBALE
# ═══════════════════════════════════════════════════════════════════════════
print("\n🌲 Isolation Forest (anomalie globali)...")

iso = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
iso_pred = iso.fit_predict(X_scaled)
iso_score = iso.decision_function(X_scaled)

df['iso_anomaly'] = (iso_pred == -1).astype(int)
df['iso_score'] = iso_score

n_anomalies = df['iso_anomaly'].sum()
print(f"\n⚠️ Anomalie globali: {n_anomalies} clienti ({n_anomalies/len(df)*100:.1f}%)")

# Distribuzione anomalie per cluster
print("\n📊 Anomalie per segmento:")
anomaly_by_segment = df.groupby('segment_name')['iso_anomaly'].agg(['sum', 'mean'])
anomaly_by_segment.columns = ['N Anomalie', '% Anomalie']
anomaly_by_segment['% Anomalie'] = anomaly_by_segment['% Anomalie'] * 100
print(anomaly_by_segment.round(1))

# ═══════════════════════════════════════════════════════════════════════════
# 7.2 LOF PER ANOMALIE LOCALI
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("🔍 Local Outlier Factor (anomalie locali)...")

lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
lof_pred = lof.fit_predict(X_scaled)
lof_score = -lof.negative_outlier_factor_

df['lof_anomaly'] = (lof_pred == -1).astype(int)
df['lof_score'] = lof_score

n_lof_anomalies = df['lof_anomaly'].sum()
print(f"\n⚠️ Anomalie LOF: {n_lof_anomalies} clienti ({n_lof_anomalies/len(df)*100:.1f}%)")

# ═══════════════════════════════════════════════════════════════════════════
# 7.3 CONSENSUS ANOMALIES
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("🤝 Consensus Anomalies (entrambi gli algoritmi)...")

df['consensus_anomaly'] = ((df['iso_anomaly'] == 1) & (df['lof_anomaly'] == 1)).astype(int)
n_consensus = df['consensus_anomaly'].sum()
print(f"\n⚠️ Anomalie concordi: {n_consensus} clienti")

# ═══════════════════════════════════════════════════════════════════════════
# 7.4 ANALISI ANOMALIE
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("📊 PROFILO CLIENTI ANOMALI")
print("-" * 70)

# Confronto anomali vs normali
normal_mask = df['iso_anomaly'] == 0
anomaly_mask = df['iso_anomaly'] == 1

comparison = pd.DataFrame({
    'Normali': df.loc[normal_mask, features_to_use].mean(),
    'Anomali': df.loc[anomaly_mask, features_to_use].mean()
})
comparison['Diff %'] = ((comparison['Anomali'] - comparison['Normali']) / comparison['Normali'] * 100)
print(comparison.round(2))

# ═══════════════════════════════════════════════════════════════════════════
# 7.5 VISUALIZZAZIONE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Anomalie in spazio PCA
ax1 = axes[0, 0]
normal = df['iso_anomaly'] == 0
ax1.scatter(X_pca_2d[normal, 0], X_pca_2d[normal, 1], c='blue', alpha=0.3, s=20, label='Normale')
ax1.scatter(X_pca_2d[~normal, 0], X_pca_2d[~normal, 1], c='red', alpha=0.8, s=50, marker='X', label='Anomalia')
ax1.set_xlabel('PC1')
ax1.set_ylabel('PC2')
ax1.set_title('🔍 Anomalie in Spazio PCA')
ax1.legend()

# Plot 2: ISO vs LOF scores
ax2 = axes[0, 1]
colors = ['red' if c == 1 else 'blue' for c in df['consensus_anomaly']]
ax2.scatter(df['iso_score'], df['lof_score'], c=colors, alpha=0.5, s=20)
ax2.set_xlabel('Isolation Forest Score')
ax2.set_ylabel('LOF Score')
ax2.set_title('📊 ISO vs LOF Scores (rosso = consensus)')
ax2.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax2.axhline(y=np.percentile(df['lof_score'], 95), color='gray', linestyle='--', alpha=0.5)

# Plot 3: Anomalie per segmento
ax3 = axes[1, 0]
segments = df['segment_name'].unique()
n_normal = []
n_anomaly = []
for seg in segments:
    mask = df['segment_name'] == seg
    n_normal.append((df.loc[mask, 'iso_anomaly'] == 0).sum())
    n_anomaly.append((df.loc[mask, 'iso_anomaly'] == 1).sum())

x = np.arange(len(segments))
width = 0.35
ax3.bar(x - width/2, n_normal, width, label='Normali', color='steelblue')
ax3.bar(x + width/2, n_anomaly, width, label='Anomalie', color='red')
ax3.set_xlabel('Segmento')
ax3.set_ylabel('N Clienti')
ax3.set_xticks(x)
ax3.set_xticklabels(segments, rotation=45, ha='right')
ax3.set_title('📊 Distribuzione Anomalie per Segmento')
ax3.legend()

# Plot 4: Box plot monetary anomali vs normali
ax4 = axes[1, 1]
df.boxplot(column='monetary', by='iso_anomaly', ax=ax4)
ax4.set_xticklabels(['Normale', 'Anomalia'])
ax4.set_xlabel('Tipo')
ax4.set_ylabel('Monetary (€)')
ax4.set_title('💰 Monetary: Normali vs Anomalie')
plt.suptitle('')

plt.tight_layout()
plt.show()

# Top 5 anomalie
print("\n📋 Top 5 Anomalie (per score):")
top_anomalies = df[df['iso_anomaly'] == 1].nsmallest(5, 'iso_score')[
    ['customer_id', 'segment_name'] + features_to_use + ['iso_score']
]
print(top_anomalies.round(2))

print("\n" + "=" * 70)
print("✅ FASE 7 COMPLETATA!")
print("=" * 70)

In [ ]:
# =============================================================================
# 📊 FASE 8: Validazione con Approcci Alternativi
# =============================================================================
"""
OBIETTIVO: Validare i risultati con altri algoritmi di clustering
"""

from sklearn.cluster import AgglomerativeClustering, DBSCAN
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

print("=" * 70)
print("📊 FASE 8: VALIDAZIONE CON APPROCCI ALTERNATIVI")
print("=" * 70)

# ═══════════════════════════════════════════════════════════════════════════
# 8.1 CLUSTERING GERARCHICO
# ═══════════════════════════════════════════════════════════════════════════
print("\n🌳 Clustering Gerarchico (Agglomerative)...")

agg = AgglomerativeClustering(n_clusters=k_optimal, linkage='ward')
agg_labels = agg.fit_predict(X_scaled)

# Confronto con K-Means
ari_agg = adjusted_rand_score(cluster_labels, agg_labels)
nmi_agg = normalized_mutual_info_score(cluster_labels, agg_labels)

print(f"\n📊 Accordo con K-Means:")
print(f"  Adjusted Rand Index: {ari_agg:.3f}")
print(f"  Normalized Mutual Information: {nmi_agg:.3f}")

# ═══════════════════════════════════════════════════════════════════════════
# 8.2 DBSCAN
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("🎯 DBSCAN...")

# Stima eps con k-distance
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5)
nn.fit(X_scaled)
distances, _ = nn.kneighbors(X_scaled)
k_distances = np.sort(distances[:, -1])

# Eps stimato dal "gomito"
eps_estimated = np.percentile(k_distances, 95)
print(f"  Eps stimato: {eps_estimated:.2f}")

dbscan = DBSCAN(eps=eps_estimated, min_samples=5)
dbscan_labels = dbscan.fit_predict(X_scaled)

n_clusters_dbscan = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = (dbscan_labels == -1).sum()

print(f"\n📊 DBSCAN Results:")
print(f"  N cluster trovati: {n_clusters_dbscan}")
print(f"  Noise points: {n_noise} ({n_noise/len(dbscan_labels)*100:.1f}%)")

# ═══════════════════════════════════════════════════════════════════════════
# 8.3 CONFRONTO VISUALE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# K-Means
ax1 = axes[0]
scatter1 = ax1.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=cluster_labels, cmap='viridis', alpha=0.6, s=30)
ax1.set_xlabel('PC1')
ax1.set_ylabel('PC2')
ax1.set_title(f'🎯 K-Means (K={k_optimal})')
plt.colorbar(scatter1, ax=ax1)

# Hierarchical
ax2 = axes[1]
scatter2 = ax2.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=agg_labels, cmap='viridis', alpha=0.6, s=30)
ax2.set_xlabel('PC1')
ax2.set_ylabel('PC2')
ax2.set_title(f'🌳 Hierarchical (K={k_optimal})')
plt.colorbar(scatter2, ax=ax2)

# DBSCAN
ax3 = axes[2]
colors_db = ['gray' if l == -1 else plt.cm.viridis(l / max(dbscan_labels.max(), 1)) for l in dbscan_labels]
ax3.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=colors_db, alpha=0.6, s=30)
ax3.set_xlabel('PC1')
ax3.set_ylabel('PC2')
ax3.set_title(f'🎯 DBSCAN (K={n_clusters_dbscan}, noise={n_noise})')

plt.tight_layout()
plt.show()

print("\n✅ I risultati K-Means sono confermati dal clustering gerarchico!")

print("\n" + "=" * 70)
print("✅ FASE 8 COMPLETATA!")
print("=" * 70)

In [ ]:
# =============================================================================
# 📋 FASE 9: Interpretazione Business e Raccomandazioni
# =============================================================================
"""
OBIETTIVO: Tradurre i risultati in insight actionable per il business
"""

print("=" * 70)
print("📋 FASE 9: INTERPRETAZIONE BUSINESS E RACCOMANDAZIONI")
print("=" * 70)

# ═══════════════════════════════════════════════════════════════════════════
# 9.1 SUMMARY SEGMENTI
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("📊 SUMMARY FINALE SEGMENTI")
print("-" * 70)

# Calcolo metriche per ogni segmento
segment_summary = df.groupby('segment_name').agg({
    'customer_id': 'count',
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': ['mean', 'sum'],
    'avg_basket': 'mean',
    'tenure': 'mean',
    'online_ratio': 'mean',
    'iso_anomaly': 'sum'
}).round(2)

segment_summary.columns = ['N_Clienti', 'Recency_Medio', 'Frequency_Media', 
                           'Monetary_Medio', 'Monetary_Totale', 'Basket_Medio',
                           'Tenure_Medio', 'Online_Ratio', 'N_Anomalie']

# Calcolo % del fatturato
segment_summary['%_Fatturato'] = (segment_summary['Monetary_Totale'] / 
                                   segment_summary['Monetary_Totale'].sum() * 100).round(1)

print(segment_summary)

# ═══════════════════════════════════════════════════════════════════════════
# 9.2 RACCOMANDAZIONI PER SEGMENTO
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("💡 RACCOMANDAZIONI STRATEGICHE")
print("-" * 70)

recommendations = {
    '🌟 VIP': {
        'Descrizione': 'Clienti premium con alta frequenza e spesa',
        'Strategia': 'RETAIN - Mantenere loyalty',
        'Azioni': [
            '✅ Programma VIP esclusivo',
            '✅ Early access nuovi prodotti',
            '✅ Customer success manager dedicato',
            '✅ Offerte personalizzate premium'
        ]
    },
    '💼 Regular': {
        'Descrizione': 'Clienti attivi con potenziale di crescita',
        'Strategia': 'GROW - Aumentare valore',
        'Azioni': [
            '✅ Cross-selling prodotti complementari',
            '✅ Programma referral con incentivi',
            '✅ Upgrade a VIP con soglie raggiungibili',
            '✅ Newsletter personalizzate'
        ]
    },
    '🎯 Occasional': {
        'Descrizione': 'Acquisti sporadici, basso engagement',
        'Strategia': 'ACTIVATE - Aumentare frequenza',
        'Azioni': [
            '✅ Campagne riattivazione con sconto',
            '✅ Reminder carrello abbandonato',
            '✅ Offerte time-limited',
            '✅ Sondaggio preferenze'
        ]
    },
    '⚠️ At Risk': {
        'Descrizione': 'Clienti inattivi a rischio churn',
        'Strategia': 'WIN-BACK - Riconquistare',
        'Azioni': [
            '✅ Campagna win-back aggressiva',
            '✅ Survey motivi inattività',
            '✅ Offerta "ci manchi" con super sconto',
            '✅ Considerare costo/beneficio retention'
        ]
    }
}

for segment, rec in recommendations.items():
    print(f"\n{segment}")
    print(f"  📝 {rec['Descrizione']}")
    print(f"  🎯 Strategia: {rec['Strategia']}")
    print(f"  📋 Azioni:")
    for action in rec['Azioni']:
        print(f"      {action}")

# ═══════════════════════════════════════════════════════════════════════════
# 9.3 ANALISI ANOMALIE - CASI SPECIALI
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("⚠️ GESTIONE ANOMALIE")
print("-" * 70)

print("""
🔍 Clienti anomali identificati richiedono attenzione speciale:

1. SUPER SPENDITORI ANOMALI (monetary >> media segmento)
   → Verificare autenticità transazioni
   → Potenziali candidati per upgrade VIP+
   → Analisi comportamento acquisto

2. FREQUENZA ANOMALA (pattern irregolari)
   → Verificare account fraud
   → Analisi temporale acquisti
   → Possibili multi-account

3. INATTIVITÀ ESTREMA (recency >>> media At Risk)
   → Candidati per rimozione da mailing list
   → Ultimo tentativo win-back
   → Archivio clienti "dormienti"
""")

# ═══════════════════════════════════════════════════════════════════════════
# 9.4 KPI E METRICHE DI SUCCESSO
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("📊 KPI PER MONITORAGGIO")
print("-" * 70)

print("""
📈 KPI da tracciare per ogni segmento:

| Segmento    | KPI Primario        | KPI Secondario     | Target        |
|-------------|---------------------|--------------------| --------------|
| 🌟 VIP      | Retention Rate      | NPS Score          | >95% / >70    |
| 💼 Regular  | Upgrade Rate a VIP  | Avg Basket Growth  | >5% / +10%    |
| 🎯 Occasional| Activation Rate    | Frequency Growth   | >20% / +50%   |
| ⚠️ At Risk  | Win-back Rate       | Reactivation Cost  | >10% / <€20   |
""")

print("\n" + "=" * 70)
print("✅ FASE 9 COMPLETATA!")
print("=" * 70)

In [ ]:
# =============================================================================
# 📦 FASE 10: Deliverables Finali
# =============================================================================
"""
OBIETTIVO: Preparare output pronti per il business
"""

print("=" * 70)
print("📦 FASE 10: DELIVERABLES FINALI")
print("=" * 70)

# ═══════════════════════════════════════════════════════════════════════════
# 10.1 DATASET ARRICCHITO FINALE
# ═══════════════════════════════════════════════════════════════════════════
print("\n📋 Dataset finale con segmentazione...")

# Preparo dataset finale
df_final = df[['customer_id', 'segment_name', 'cluster'] + features_to_use + 
              ['iso_anomaly', 'iso_score']].copy()

# Aggiungo distanza dal centroide (proxy per "tipicità")
distances_to_centroid = cdist(X_scaled, kmeans.cluster_centers_)
df_final['dist_to_centroid'] = [distances_to_centroid[i, cluster_labels[i]] 
                                 for i in range(len(df))]

# Aggiungo componenti PCA
df_final['PC1'] = X_pca_2d[:, 0]
df_final['PC2'] = X_pca_2d[:, 1]

print(f"\n📐 Shape dataset finale: {df_final.shape}")
print(f"\n📋 Colonne:")
for col in df_final.columns:
    print(f"  • {col}")

# ═══════════════════════════════════════════════════════════════════════════
# 10.2 EXPORT FUNZIONE PREDIZIONE
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("🔧 FUNZIONE DI SCORING PER NUOVI CLIENTI")
print("-" * 70)

def score_new_customer(customer_data, scaler, kmeans, iso_forest, pca, cluster_names):
    """
    Assegna un nuovo cliente al segmento appropriato.
    
    Args:
        customer_data: dict con features del cliente
        scaler: RobustScaler fittato
        kmeans: modello K-Means fittato
        iso_forest: modello IsolationForest fittato
        pca: modello PCA fittato
        cluster_names: dizionario mapping cluster -> nome
        
    Returns:
        dict con risultati scoring
    """
    # Prepara dati
    X_new = pd.DataFrame([customer_data])[features_to_use]
    
    # Scala
    X_scaled_new = scaler.transform(X_new)
    
    # Cluster prediction
    cluster = kmeans.predict(X_scaled_new)[0]
    segment = cluster_names[cluster]
    
    # Distanza dal centroide
    dist = np.linalg.norm(X_scaled_new - kmeans.cluster_centers_[cluster])
    
    # Anomaly score
    anomaly_score = iso_forest.decision_function(X_scaled_new)[0]
    is_anomaly = iso_forest.predict(X_scaled_new)[0] == -1
    
    # PCA coordinates
    pca_coords = pca.transform(X_scaled_new)[0]
    
    return {
        'segment': segment,
        'cluster_id': cluster,
        'distance_to_centroid': dist,
        'is_anomaly': is_anomaly,
        'anomaly_score': anomaly_score,
        'pc1': pca_coords[0],
        'pc2': pca_coords[1]
    }

# Test con un cliente esempio
test_customer = {
    'recency': 10,
    'frequency': 25,
    'monetary': 6000,
    'avg_basket': 100,
    'tenure': 36,
    'online_ratio': 0.7
}

result = score_new_customer(test_customer, scaler, kmeans, iso, pca_2d, cluster_names)
print("\n📋 Test scoring nuovo cliente:")
print(f"  Input: {test_customer}")
print(f"\n  Output:")
for k, v in result.items():
    print(f"    {k}: {v}")

# ═══════════════════════════════════════════════════════════════════════════
# 10.3 DASHBOARD SUMMARY
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-" * 70)
print("📊 DASHBOARD SUMMARY")
print("-" * 70)

# Metriche chiave
total_customers = len(df)
total_revenue = df['monetary'].sum()
avg_clv = df['monetary'].mean()
vip_pct = (df['segment_name'] == '🌟 VIP').mean() * 100
at_risk_pct = (df['segment_name'] == '⚠️ At Risk').mean() * 100
anomaly_pct = df['iso_anomaly'].mean() * 100

print(f"""
┌─────────────────────────────────────────────────────────────────────┐
│                    📊 CUSTOMER SEGMENTATION DASHBOARD                │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  👥 TOTAL CUSTOMERS: {total_customers:,}                                        │
│  💰 TOTAL REVENUE: €{total_revenue:,.0f}                                  │
│  📈 AVG CLV: €{avg_clv:,.0f}                                              │
│                                                                      │
├─────────────────────────────────────────────────────────────────────┤
│  SEGMENTATION                                                        │
│  ────────────────────────────────────────────                        │
│  🌟 VIP:        {(df['segment_name'] == '🌟 VIP').sum():3d} ({vip_pct:4.1f}%)                                    │
│  💼 Regular:    {(df['segment_name'] == '💼 Regular').sum():3d} ({(df['segment_name'] == '💼 Regular').mean()*100:4.1f}%)                                    │
│  🎯 Occasional: {(df['segment_name'] == '🎯 Occasional').sum():3d} ({(df['segment_name'] == '🎯 Occasional').mean()*100:4.1f}%)                                    │
│  ⚠️ At Risk:    {(df['segment_name'] == '⚠️ At Risk').sum():3d} ({at_risk_pct:4.1f}%)                                    │
│                                                                      │
├─────────────────────────────────────────────────────────────────────┤
│  ⚠️ ANOMALIES: {df['iso_anomaly'].sum():3d} ({anomaly_pct:4.1f}%)                                        │
└─────────────────────────────────────────────────────────────────────┘
""")

# ═══════════════════════════════════════════════════════════════════════════
# 10.4 VISUALIZZAZIONE FINALE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Plot 1: Pie chart segmenti
ax1 = axes[0, 0]
segment_counts = df['segment_name'].value_counts()
colors_pie = ['gold', 'steelblue', 'green', 'red']
ax1.pie(segment_counts.values, labels=segment_counts.index, autopct='%1.1f%%',
        colors=colors_pie, explode=[0.05, 0, 0, 0], shadow=True)
ax1.set_title('📊 Distribuzione Segmenti')

# Plot 2: Revenue per segment
ax2 = axes[0, 1]
revenue_by_segment = df.groupby('segment_name')['monetary'].sum().sort_values(ascending=True)
revenue_by_segment.plot(kind='barh', ax=ax2, color=['red', 'green', 'steelblue', 'gold'])
ax2.set_xlabel('Revenue Totale (€)')
ax2.set_title('💰 Revenue per Segmento')
for i, v in enumerate(revenue_by_segment.values):
    ax2.text(v + 1000, i, f'€{v:,.0f}', va='center')

# Plot 3: Segmenti in spazio PCA con centroidi
ax3 = axes[1, 0]
for c in range(k_optimal):
    mask = cluster_labels == c
    ax3.scatter(X_pca_2d[mask, 0], X_pca_2d[mask, 1], 
                c=colors_pie[c], label=cluster_names[c], alpha=0.6, s=30)
# Centroidi
ax3.scatter(centroids_pca[:, 0], centroids_pca[:, 1], 
            c='black', marker='X', s=300, edgecolors='white', linewidths=2, zorder=5)
ax3.set_xlabel('PC1')
ax3.set_ylabel('PC2')
ax3.set_title('🎯 Mappa Segmenti Clienti')
ax3.legend(loc='upper right')

# Plot 4: RFM 3D plot
ax4 = fig.add_subplot(2, 2, 4, projection='3d')
for c in range(k_optimal):
    mask = df['cluster'] == c
    ax4.scatter(df.loc[mask, 'recency'], df.loc[mask, 'frequency'], 
                df.loc[mask, 'monetary'], c=colors_pie[c], 
                label=cluster_names[c], alpha=0.5, s=20)
ax4.set_xlabel('Recency')
ax4.set_ylabel('Frequency')
ax4.set_zlabel('Monetary')
ax4.set_title('📊 RFM 3D View')

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("✅ FASE 10 COMPLETATA!")
print("=" * 70)

---

# 🎯 CONCLUSIONE PROGETTO

## 🏆 Risultati Ottenuti

| Obiettivo | Risultato | Tecnica Usata |
|-----------|-----------|---------------|
| **Quanti segmenti?** | 4 segmenti naturali | K-Means + Elbow + Silhouette |
| **Come si caratterizzano?** | VIP, Regular, Occasional, At Risk | Profiling + Radar Chart |
| **Chi sono gli anomali?** | ~5% del dataset | Isolation Forest + LOF |
| **Strategia per segmento?** | Retain, Grow, Activate, Win-back | Business Interpretation |

## 📊 Tecniche Utilizzate (Lezioni 19-27)

| Lezione | Tecnica | Applicazione |
|---------|---------|--------------|
| 19 | Unsupervised Intro | Framework mentale |
| 20 | K-Means | Segmentazione primaria |
| 21 | Scelta K | Elbow + Silhouette + CH + DB |
| 22 | Hierarchical | Validazione alternativa |
| 23 | DBSCAN | Noise detection naturale |
| 24 | PCA | Riduzione 6D → 2D |
| 25 | PCA + Clustering | Visualizzazione segmenti |
| 26 | Anomaly Detection | IsolationForest + LOF |
| 27 | Feature Engineering | Distanze, scores, profili |

## 💡 Lesson Learned

1. **Preprocessing è fondamentale** — RobustScaler per outlier
2. **Validare con multiple metriche** — Non affidarsi a una sola
3. **Cross-validare con algoritmi diversi** — Gerarchico conferma K-Means
4. **Business context guida le decisioni** — 4 segmenti interpretabili
5. **Anomalie richiedono attenzione speciale** — Non sono solo errori

---

# 📚 BIGNAMI — Unsupervised Learning Completo

## 🔧 Pipeline Tipo Progetto Unsupervised

```
1. BUSINESS UNDERSTANDING
   └── Definire obiettivi e domande

2. DATA LOADING & EDA
   └── Caricare, esplorare, visualizzare

3. PREPROCESSING
   ├── Handling missing values
   ├── Outlier detection
   └── Scaling (StandardScaler/RobustScaler)

4. DIMENSIONALITY REDUCTION
   ├── PCA per visualizzazione
   └── Scegliere n_components (80-90% varianza)

5. SCELTA K
   ├── Elbow Method
   ├── Silhouette Score
   ├── Calinski-Harabasz
   └── Davies-Bouldin

6. CLUSTERING
   ├── K-Means (principale)
   ├── Hierarchical (validazione)
   └── DBSCAN (noise detection)

7. ANOMALY DETECTION
   ├── Isolation Forest
   └── Local Outlier Factor

8. VALIDATION
   ├── Cross-validation con altri algoritmi
   └── Metriche: ARI, NMI

9. INTERPRETATION
   ├── Profiling cluster
   ├── Business naming
   └── Raccomandazioni actionable

10. DELIVERABLES
    ├── Dataset arricchito
    ├── Funzione scoring
    └── Dashboard/Report
```

## 📊 Metriche Chiave

| Metrica | Formula/Comando | Ottimo |
|---------|-----------------|--------|
| Silhouette | `silhouette_score(X, labels)` | Max (-1 to 1) |
| Calinski-Harabasz | `calinski_harabasz_score(X, labels)` | Max |
| Davies-Bouldin | `davies_bouldin_score(X, labels)` | Min |
| Inertia | `kmeans.inertia_` | Gomito |

## 🔧 Algoritmi Quick Reference

```python
# K-Means
from sklearn.cluster import KMeans
km = KMeans(n_clusters=4, random_state=42, n_init=10)
labels = km.fit_predict(X)

# PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Isolation Forest
from sklearn.ensemble import IsolationForest
iso = IsolationForest(contamination=0.05)
anomalies = iso.fit_predict(X)

# LOF
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(n_neighbors=20)
anomalies = lof.fit_predict(X)
```

---

# 🎉 FINE BLOCCO UNSUPERVISED LEARNING!

**Congratulazioni!** Hai completato il percorso completo di Unsupervised Learning:

- ✅ Lezione 19: Introduzione
- ✅ Lezione 20: K-Means
- ✅ Lezione 21: Scelta K
- ✅ Lezione 22: Clustering Gerarchico
- ✅ Lezione 23: DBSCAN
- ✅ Lezione 24: PCA
- ✅ Lezione 25: PCA + Clustering
- ✅ Lezione 26: Anomaly Detection
- ✅ Lezione 27: Feature Engineering
- ✅ Lezione 28: Progetto End-to-End

---

**🚀 Prossimi passi consigliati:**
1. Applicare queste tecniche a dataset reali
2. Esplorare altri algoritmi (HDBSCAN, t-SNE, UMAP)
3. Integrare con Supervised Learning (semi-supervised)
4. Studiare Deep Learning per Unsupervised (Autoencoders, VAE)